# 알바천국 크롤러

In [2]:
#-*- coding:utf-8 -*-

from bs4 import BeautifulSoup
import urllib.request
import csv
import datetime

### 한페이지에 알바 50개 보여주는 알바몬 url주소 하나를 넣으면  
### [알바 이름, 알바 설명, 알바지역, 급여종류(시급일급 등), 급여, 노동시간, 각 알바의 상세 주소] 를 뽑아주는 함수

In [4]:
def albaheaven_info_get(a_url):
    
    with urllib.request.urlopen(a_url) as url:  
        doc = url.read() #페이지에 있는 내용 가져와서 string 형식으로 저장해줌
    alba_doc = BeautifulSoup(doc, "html.parser")
    
    if len(alba_doc.find_all('tbody')) != 1:
        tbody = alba_doc.find_all('tbody')[1]
    else:
        tbody = alba_doc.find_all('tbody')[0]
    
    summaries = tbody.find_all(class_ = "summaryView")
    tlist = tbody.find_all('tr')
    
    alba_info_onepage = []
    base_url = "http://www.alba.co.kr"
    
    for alba in tlist:
        if alba in summaries:
            pass
        else:
            a_name = alba.find('span', class_='company').text
            a_juso = base_url+str(alba.a).split("\"")[3]
            a_text = alba.find('span', class_='title').text
        
            not_area = ['재택근무', '전국']
            not_area_bool = False
        
            for n in not_area:
                area = alba.find('td', class_='local').text.strip()
                if (n in area) & (not_area_bool == False):
                    a_area = [n, n]    
                    not_area_bool = True
                else:
                    a_area = area.split()[0:2] # [시도, 구]
#                 a_area_B = a_area[0]  #Big 단위
#                 a_area_S = a_area[1]  #Small 단위

            a_paytype = alba.find('td', class_='pay').find('span').text.strip()
            a_paywon = int(alba.find('td', class_ = "pay").find('span', class_='number').text.replace(',', ''))
            a_worktime = alba.find('td', class_ = "data").text[0:11]

            alba_info_onepage.append([a_name, a_text, a_area, a_paytype, a_paywon, a_worktime, a_juso])
    return alba_info_onepage

In [5]:
# 잘 되는지 test
test = albaheaven_info_get("http://www.alba.co.kr/job/period/Short.asp?page=2&pagesize=50")

In [6]:
len(test)

50

In [7]:
test

[['KD',
  '12.15일요일/일당 당일지급/야간 택배 알바/택배 상하차/대전 신탄진 허브',
  ['경기', '수원시'],
  '일급',
  150000,
  '19:00~07:30',
  'http://www.alba.co.kr/job/Detail.asp?adid=97431751'],
 ['쿠팡헬퍼-장단기알바/굿윌아이엔씨(주)',
  '(주급지급/12월19일 저녁출근) 쿠팡 시흥캠프 1W(C)근무 사원모집',
  ['서울', '구로구'],
  '일급',
  31500,
  '18:30~22:00',
  'http://www.alba.co.kr/job/Detail.asp?adid=97420351'],
 ['아이앤지유피플',
  '시급8,350/월,화/동대문역사문화공원역/반얀트리호텔/연회서빙/하루가능',
  ['서울', '전체'],
  '시급',
  8350,
  '13:00~22:00',
  'http://www.alba.co.kr/job/Detail.asp?adid=97414616'],
 ['쿠팡 풀필먼트 서비스',
  '[안성쿠팡] 쿠팡 Let it go/꿀알바/즉시출근/익일지급/단기/장기',
  ['경기', '안산시'],
  '일급',
  86093,
  '16:30~01:30',
  'http://www.alba.co.kr/job/Detail.asp?adid=97242350'],
 ['[안성쿠팡 E팀]',
  '안성쿠팡 E팀 [바로출근][주말] 꿀알바/초보/단순업무/단기/장기/지게차',
  ['경기', '안성시'],
  '일급',
  86093,
  '16:30~01:30',
  'http://www.alba.co.kr/job/Detail.asp?adid=97430443'],
 ['호텔신라',
  '[시급11,000원]신라호텔 배송도우미 모집(주휴포함)',
  ['경기', '성남시'],
  '시급',
  11000,
  '08:00~17:00',
  'http://www.alba.co.kr/job/Detail.asp?adid=97344582'],

### 1~page limit(지정된 변수)까지 페이지를 돌면서 각 페이지의 알바정보(50개)를 저장해주는 함수

##### 한 페이지 돌 때 마다 1초씩 쉬어야 악성코드로 판별되지 않고 잘 수집됨

In [11]:
import time

#nobar = "http://www.albamon.com/list/gi/mon_icon_list.asp?itype=11&ps=50&sExcChk=y"

def alba_page_iter(page_limit):
    page_n = 1
    base_url_top = "http://www.alba.co.kr/job/period/Short.asp?page="
    base_url_bottom = "&pagesize=50"
#     nobar = "&sExcChk=y"

    alba_info_whole = []
    
    while page_n < page_limit:
        search_url = base_url_top + str(page_n) + base_url_bottom #+ nobar
        print(search_url)
        a_list = albaheaven_info_get(search_url)
        for alba in a_list:
            alba_info_whole.append(alba)            
        page_n += 1
        time.sleep(1)
    
    return alba_info_whole

# 아래에서는 60페이지만큼 구해보았다. 50페이지 이후로 내용없으므로 3000건 전후로 예상.

In [14]:
alba_list = alba_page_iter(60)

http://www.alba.co.kr/job/period/Short.asp?page=1&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=2&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=3&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=4&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=5&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=6&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=7&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=8&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=9&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=10&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=11&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=12&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=13&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=14&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=15&pagesize=50
http://www.alba.co.kr/job/period/Short.asp?page=16&pagesize=50
h

In [15]:
len(alba_list)

2665

In [16]:
# 재택근무나 전국의 경우 형태를 다른 위치정보와 동일하게 ['재택근무', '재택근무']로 바꿔줄 필요가 있음. 


for i in range(len(alba_list)):
    if len(alba_list[i][2]) == 1:
        alba_list[i][2].append(alba_list[i][2][0])
        

## csv파일로 저장


In [18]:
import csv
 
f = open('albaheaven_info_1215.csv', 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
wr.writerow(['알바명', '알바설명', '지역', '지역세부', '급여종류', '금액', '시간', '주소'])

a = alba_list
for i in range(len(a)):
    wr.writerow([a[i][0], a[i][1], a[i][2][0], a[i][2][1], a[i][3],  a[i][4], a[i][5], a[i][6]]) 

f.close()